In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import numpy as np
import pandas as pd
import admix
import dask
import dask.array as da
from tqdm import tqdm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from os.path import join
import os
import admix_genet_cor

In [2]:
DATA_DIR = (
    "/u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/02-gwas/out/aframr"
)

In [3]:
SUPP_TABLE_URL = "https://www.dropbox.com/s/jck2mhjby2ur55j/supp_tables.xlsx?dl=1"
trait_info = pd.read_excel(SUPP_TABLE_URL, sheet_name="trait-tmp")
trait_list = trait_info["trait"].values

In [21]:
df_clump_list = []
for trait in trait_list:
    clumped_file = join(DATA_DIR, trait, "PLINK.imputed.clumped")
    if os.path.exists(clumped_file):
        with open(clumped_file) as f:
            lines = f.readlines()
            if len(lines) == 1:
                continue
        df_clump = pd.read_csv(clumped_file, delim_whitespace=True)[
            ["CHR", "SNP", "BP"]
        ].rename(columns={"CHR": "CHROM", "BP": "POS"})
        df_clump.insert(0, "trait", trait)
        df_clump_list.append(df_clump)
df_clump = pd.concat(df_clump_list)
df_clump.index = (
    df_clump["trait"]
    + ":"
    + df_clump["CHROM"].astype(str)
    + ":"
    + df_clump["POS"].astype(str)
)
df_clump.to_csv("out/page-gwas-insample.tsv", sep="\t")